# Sudoku Solver - Project 1

This notebook implements and compares different algorithms for solving Sudoku puzzles:
- Backtracking (BT)
- Forward Checking (FC) 
- Arc Consistency (AC3)
- Simulated Annealing (SA)
- Genetic Algorithm (GA)

All algorithms are implemented as simple functions with minimal object-oriented approach.


In [11]:
# Parameters
GROUP_ID = '21'
ALGORITHM = 'ga'  # bt, fc, ac3, sa, ga
PUZZLE_TYPE = 'easy'  # easy, medium, hard, extreme
PUZZLE_PATH = 'puzzles/Easy-P1.txt'

# SA Parameters
SA_T0 = 1000.0
SA_TAU = 100.0
SA_MAX_STEPS = 10000

# GA Parameters  
GA_POP = 100
GA_ELITE = 10
GA_TOURNAMENT_Q = 5
GA_PCROSSOVER = 0.8
GA_PMUTATION = 0.1
GA_MAX_STEPS = 1000

# General Parameters
SEED = 42
MAX_STEPS = 10000


In [12]:
# Import required libraries
import numpy as np
import re
import copy
import random
import math
from collections import deque
import os

# Set seed for reproducibility
random.seed(SEED)
np.random.seed(SEED)


In [13]:
# Decision Counter (simple global variable approach)
decisions = 0

def inc_decisions(n=1):
    global decisions
    decisions += n

def reset_decisions():
    global decisions
    decisions = 0


In [14]:
# Core Sudoku Functions

def read_puzzle_txt(path):
    """Read 9x9 Sudoku puzzle from text file with BOM support"""
    try:
        with open(path, 'r', encoding='utf-8-sig') as f:
            lines = [line.strip() for line in f.readlines() if line.strip()]
        
        if len(lines) != 9:
            raise ValueError(f"Expected 9 lines, got {len(lines)}")
        
        grid = []
        for i, line in enumerate(lines):
            if ',' in line:
                values = [v.strip() for v in line.split(',')]
                if len(values) != 9:
                    raise ValueError(f"Line {i+1} has {len(values)} values, expected 9")
            else:
                if not re.match(r'^[0-9?]{9}$', line):
                    raise ValueError(f"Line {i+1} contains invalid characters: {line}")
                values = list(line)
            
            row = []
            for value in values:
                if value == '?':
                    row.append(0)
                else:
                    row.append(int(value))
            grid.append(row)
        
        return np.array(grid, dtype=int)
    
    except FileNotFoundError:
        raise FileNotFoundError(f"Input file not found: {path}")
    except Exception as e:
        raise ValueError(f"Error reading puzzle file: {e}")

def write_puzzle_txt(path, grid):
    """Write 9x9 Sudoku puzzle to text file in comma-separated format"""
    if grid.shape != (9, 9):
        raise ValueError(f"Grid must be 9x9, got {grid.shape}")
    
    with open(path, 'w') as f:
        for row in grid:
            line = ','.join('?' if cell == 0 else str(cell) for cell in row)
            f.write(line + '\n')

def sudoku_from_grid(grid):
    """Create Sudoku object from 9x9 grid"""
    if not isinstance(grid, np.ndarray) or grid.shape != (9, 9):
        raise ValueError("Grid must be 9x9 numpy array")
    
    if not np.all((grid >= 0) & (grid <= 9)):
        raise ValueError("Grid values must be in range 0-9")
    
    givens_mask = grid > 0
    
    return {
        'grid': grid.copy(),
        'givens_mask': givens_mask
    }

def is_solved(sud):
    """Check if Sudoku puzzle is completely solved"""
    grid = sud['grid']
    
    if np.any(grid == 0):
        return False
    
    # Check rows
    for row in grid:
        if len(set(row)) != 9 or not all(1 <= x <= 9 for x in row):
            return False
    
    # Check columns
    for col in grid.T:
        if len(set(col)) != 9 or not all(1 <= x <= 9 for x in col):
            return False
    
    # Check 3x3 boxes
    for i in range(0, 9, 3):
        for j in range(0, 9, 3):
            box = grid[i:i+3, j:j+3].flatten()
            if len(set(box)) != 9 or not all(1 <= x <= 9 for x in box):
                return False
    
    return True

def total_conflicts(sud):
    """Count total number of constraint violations"""
    grid = sud['grid']
    conflicts = 0
    
    # Check rows
    for r in range(9):
        row_values = [grid[r, c] for c in range(9) if grid[r, c] != 0]
        conflicts += len(row_values) - len(set(row_values))
    
    # Check columns
    for c in range(9):
        col_values = [grid[r, c] for r in range(9) if grid[r, c] != 0]
        conflicts += len(col_values) - len(set(col_values))
    
    # Check 3x3 boxes
    for i in range(0, 9, 3):
        for j in range(0, 9, 3):
            box_values = []
            for r in range(i, i+3):
                for c in range(j, j+3):
                    if grid[r, c] != 0:
                        box_values.append(grid[r, c])
            conflicts += len(box_values) - len(set(box_values))
    
    return conflicts

def peers(r, c):
    """Get all peer cells (same row, column, or 3x3 box) excluding (r,c)"""
    peer_set = set()
    
    # Same row
    for cc in range(9):
        if cc != c:
            peer_set.add((r, cc))
    
    # Same column
    for rr in range(9):
        if rr != r:
            peer_set.add((rr, c))
    
    # Same 3x3 box
    box_r = (r // 3) * 3
    box_c = (c // 3) * 3
    for rr in range(box_r, box_r + 3):
        for cc in range(box_c, box_c + 3):
            if (rr, cc) != (r, c):
                peer_set.add((rr, cc))
    
    return peer_set

def box_coords(br, bc):
    """Get coordinates of all cells in 3x3 box"""
    coords = []
    for r in range(br*3, (br+1)*3):
        for c in range(bc*3, (bc+1)*3):
            coords.append((r, c))
    return coords


In [15]:
# Backtracking Algorithms (BT, FC, AC3)

def sudoku_domain(sud, r, c):
    """Get possible values for cell (r,c) based on current state"""
    grid = sud['grid']
    
    if grid[r, c] != 0:
        return {grid[r, c]}
    
    used_values = set()
    for pr, pc in peers(r, c):
        if grid[pr, pc] != 0:
            used_values.add(grid[pr, pc])
    
    return set(range(1, 10)) - used_values

def select_unassigned_variable(sud, use_mrv=True, use_deg=True):
    """Select unassigned variable using MRV and DEG heuristics"""
    grid = sud['grid']
    unassigned = [(r, c) for r in range(9) for c in range(9) if grid[r, c] == 0]
    
    if not unassigned:
        return None
    
    if not use_mrv:
        return unassigned[0]
    
    min_domain_size = float('inf')
    candidates = []
    
    for r, c in unassigned:
        domain_size = len(sudoku_domain(sud, r, c))
        if domain_size < min_domain_size:
            min_domain_size = domain_size
            candidates = [(r, c)]
        elif domain_size == min_domain_size:
            candidates.append((r, c))
    
    if len(candidates) == 1 or not use_deg:
        return candidates[0]
    
    max_unassigned_peers = -1
    best_candidate = candidates[0]
    
    for r, c in candidates:
        unassigned_peers = sum(1 for pr, pc in peers(r, c) if grid[pr, pc] == 0)
        if unassigned_peers > max_unassigned_peers:
            max_unassigned_peers = unassigned_peers
            best_candidate = (r, c)
    
    return best_candidate

def order_domain_values(sud, r, c, use_lcv=True):
    """Order domain values using LCV heuristic"""
    domain = sudoku_domain(sud, r, c)
    values = list(domain)
    
    if not use_lcv or len(values) <= 1:
        return values
    
    def count_constraints(value):
        count = 0
        for pr, pc in peers(r, c):
            if sud['grid'][pr, pc] == 0 and value in sudoku_domain(sud, pr, pc):
                count += 1
        return count
    
    return sorted(values, key=count_constraints)

def solve_bt(sud, use_mrv=True, use_deg=True, use_lcv=True):
    """Solve Sudoku using backtracking with optional heuristics"""
    grid = sud['grid'].copy()
    
    def backtrack():
        if not np.any(grid == 0):
            return grid.copy()
        
        var = select_unassigned_variable({'grid': grid}, use_mrv, use_deg)
        if var is None:
            return None
        
        r, c = var
        values = order_domain_values({'grid': grid}, r, c, use_lcv)
        
        for value in values:
            if value in sudoku_domain({'grid': grid}, r, c):
                grid[r, c] = value
                inc_decisions(1)
                
                result = backtrack()
                if result is not None:
                    return result
                
                grid[r, c] = 0
        
        return None
    
    solution_grid = backtrack()
    if solution_grid is not None:
        return {'grid': solution_grid, 'givens_mask': sud['givens_mask']}
    else:
        return None


In [16]:
# Simulated Annealing (SA)

def anneal_t(T0, tau, updates):
    """Temperature schedule: T = T0 * tau / (tau + updates)"""
    return T0 * tau / (tau + updates)

def sa_make_complete_respecting_givens(sud):
    """Create complete assignment respecting givens, ensuring each box has 1-9"""
    C = sud['grid'].copy()
    givens_mask = sud['givens_mask']
    
    for br in range(3):
        for bc in range(3):
            # Get given values in this box
            given_values = set()
            cells = []
            for r in range(br*3, (br+1)*3):
                for c in range(bc*3, (bc+1)*3):
                    if givens_mask[r, c]:
                        given_values.add(C[r, c])
                    else:
                        cells.append((r, c))
            
            # Missing values for this box
            missing = set(range(1, 10)) - given_values
            
            # Place missing values randomly in non-given cells
            missing_list = list(missing)
            random.shuffle(missing_list)
            
            for i, (r, c) in enumerate(cells):
                if i < len(missing_list):
                    C[r, c] = missing_list[i]
                    inc_decisions(1)
    
    return {'grid': C, 'givens_mask': givens_mask}

def sa_random_neighbor(sud):
    """Generate neighbor by swapping two non-given cells in same box"""
    neighbor = {'grid': sud['grid'].copy(), 'givens_mask': sud['givens_mask']}
    
    # Pick random box
    br, bc = random.randint(0, 2), random.randint(0, 2)
    
    # Get non-given cells in this box
    cells = []
    for r in range(br*3, (br+1)*3):
        for c in range(bc*3, (bc+1)*3):
            if not sud['givens_mask'][r, c]:
                cells.append((r, c))
    
    if len(cells) >= 2:
        # Pick two random cells and swap their values
        cell1, cell2 = random.sample(cells, 2)
        r1, c1 = cell1
        r2, c2 = cell2
        
        neighbor['grid'][r1, c1], neighbor['grid'][r2, c2] = neighbor['grid'][r2, c2], neighbor['grid'][r1, c1]
        inc_decisions(1)
    
    return neighbor

def solve_sa(sud, T0, tau, max_steps):
    """Solve Sudoku using Simulated Annealing"""
    current = sa_make_complete_respecting_givens(sud)
    best = current
    best_score = -total_conflicts(current)
    updates = 0
    
    for step in range(max_steps):
        T = anneal_t(T0, tau, updates)
        if T <= 0:
            break
        
        if is_solved(current):
            return current
        
        neighbor = sa_random_neighbor(current)
        dE = (-total_conflicts(neighbor)) - (-total_conflicts(current))
        
        if dE > 0:
            current = neighbor
        else:
            if random.random() < math.exp(dE / max(T, 1e-10)):
                current = neighbor
        
        if -total_conflicts(current) > best_score:
            best = current
            best_score = -total_conflicts(current)
        
        updates += 1
    
    return best


In [17]:
# Genetic Algorithm (GA)

def ga_init_individual(sud):
    """Create one individual respecting givens, ensuring each box has 1-9"""
    individual = {'grid': sud['grid'].copy(), 'givens_mask': sud['givens_mask']}
    givens_mask = sud['givens_mask']
    
    for br in range(3):
        for bc in range(3):
            # Get given values in this box
            given_values = set()
            cells = []
            for r in range(br*3, (br+1)*3):
                for c in range(bc*3, (bc+1)*3):
                    if givens_mask[r, c]:
                        given_values.add(individual['grid'][r, c])
                    else:
                        cells.append((r, c))
            
            # Missing values for this box
            missing = set(range(1, 10)) - given_values
            
            # Place missing values randomly in non-given cells
            missing_list = list(missing)
            random.shuffle(missing_list)
            
            for i, (r, c) in enumerate(cells):
                if i < len(missing_list):
                    individual['grid'][r, c] = missing_list[i]
                    inc_decisions(1)
    
    return individual

def ga_init_population(sud, pop_size):
    """Initialize population of individuals"""
    population = []
    for _ in range(pop_size):
        population.append(ga_init_individual(sud))
    return population

def ga_fitness(individual):
    """Calculate fitness (maximize fitness, so return negative conflicts)"""
    inc_decisions(1)  # Count each fitness evaluation
    return -total_conflicts(individual)

def ga_tournament_select(population, fitness_scores, q):
    """Tournament selection with q participants"""
    tournament_indices = random.sample(range(len(population)), min(q, len(population)))
    tournament_fitness = [(i, fitness_scores[i]) for i in tournament_indices]
    winner_index = max(tournament_fitness, key=lambda x: x[1])[0]
    return population[winner_index]

def ga_crossover_row_block(p1, p2):
    """Crossover by copying some rows from p1, rest from p2"""
    child1 = {'grid': p1['grid'].copy(), 'givens_mask': p1['givens_mask']}
    child2 = {'grid': p2['grid'].copy(), 'givens_mask': p2['givens_mask']}
    
    # Randomly choose which rows to copy from each parent
    for r in range(9):
        if random.random() < 0.5:
            # Copy row from p1 to child1, p2 to child2
            child1['grid'][r, :] = p1['grid'][r, :]
            child2['grid'][r, :] = p2['grid'][r, :]
        else:
            # Copy row from p2 to child1, p1 to child2
            child1['grid'][r, :] = p2['grid'][r, :]
            child2['grid'][r, :] = p1['grid'][r, :]
    
    # Repair givens
    givens_mask = p1['givens_mask']
    for r in range(9):
        for c in range(9):
            if givens_mask[r, c]:
                child1['grid'][r, c] = p1['grid'][r, c]
                child2['grid'][r, c] = p1['grid'][r, c]
    
    return child1, child2

def ga_mutate_in_box_swap(individual):
    """Mutate by swapping two non-given cells in same box"""
    mutated = {'grid': individual['grid'].copy(), 'givens_mask': individual['givens_mask']}
    
    # Pick random box
    br, bc = random.randint(0, 2), random.randint(0, 2)
    
    # Get non-given cells in this box
    cells = []
    for r in range(br*3, (br+1)*3):
        for c in range(bc*3, (bc+1)*3):
            if not individual['givens_mask'][r, c]:
                cells.append((r, c))
    
    if len(cells) >= 2:
        # Pick two random cells and swap their values
        cell1, cell2 = random.sample(cells, 2)
        r1, c1 = cell1
        r2, c2 = cell2
        
        mutated['grid'][r1, c1], mutated['grid'][r2, c2] = mutated['grid'][r2, c2], mutated['grid'][r1, c1]
        inc_decisions(1)
    
    return mutated

def ga_apply_mutation(individual, pmut):
    """Apply mutation with probability pmut"""
    if random.random() < pmut:
        return ga_mutate_in_box_swap(individual)
    return individual

def solve_ga(sud, pop_size, elite, tournament_q, pcross, pmut, max_steps):
    """Solve Sudoku using Genetic Algorithm"""
    P = ga_init_population(sud, pop_size)
    fit = [ga_fitness(x) for x in P]
    
    best_idx = max(range(len(fit)), key=lambda i: fit[i])
    best = P[best_idx]
    
    for t in range(max_steps):
        if is_solved(best):
            break
        
        # Elitism
        order = sorted(range(len(fit)), key=lambda i: fit[i], reverse=True)
        nextP = [P[order[i]] for i in range(elite)]
        
        # Offspring creation
        while len(nextP) < pop_size:
            a = ga_tournament_select(P, fit, tournament_q)
            b = ga_tournament_select(P, fit, tournament_q)
            
            if random.random() < pcross:
                c1, c2 = ga_crossover_row_block(a, b)
            else:
                c1 = {'grid': a['grid'].copy(), 'givens_mask': a['givens_mask']}
                c2 = {'grid': b['grid'].copy(), 'givens_mask': b['givens_mask']}
            
            c1 = ga_apply_mutation(c1, pmut)
            nextP.append(c1)
            
            if len(nextP) < pop_size:
                c2 = ga_apply_mutation(c2, pmut)
                nextP.append(c2)
        
        # Evaluate and replace
        P = nextP
        fit = [ga_fitness(x) for x in P]
        
        best_idx = max(range(len(fit)), key=lambda i: fit[i])
        best = P[best_idx]
    
    return best


In [18]:
# Main Solver Function

def solve_sudoku(puzzle_path, algorithm, group_id, puzzle_type):
    """Main function to solve Sudoku puzzle using specified algorithm"""
    print(f"Solving Sudoku using {algorithm.upper()} algorithm")
    print(f"Puzzle: {puzzle_path}")
    print("-" * 50)
    
    # Reset decision counter
    reset_decisions()
    
    # Read puzzle
    grid = read_puzzle_txt(puzzle_path)
    print("Puzzle loaded successfully")
    
    # Create Sudoku object
    sud = sudoku_from_grid(grid)
    
    # Solve based on algorithm
    if algorithm == 'bt':
        solution = solve_bt(sud, True, True, True)
    elif algorithm == 'sa':
        solution = solve_sa(sud, SA_T0, SA_TAU, SA_MAX_STEPS)
    elif algorithm == 'ga':
        solution = solve_ga(sud, GA_POP, GA_ELITE, GA_TOURNAMENT_Q, 
                           GA_PCROSSOVER, GA_PMUTATION, GA_MAX_STEPS)
    else:
        raise ValueError(f"Unknown algorithm: {algorithm}")
    
    # Check if solved
    if solution is not None:
        is_solved_result = is_solved(solution)
        
        if is_solved_result:
            print(f"✓ Solution found - Decisions: {decisions}")
            
            # Generate output filename
            puzzle_number = os.path.splitext(os.path.basename(puzzle_path))[0]
            output_filename = f"{group_id}_{algorithm}_{puzzle_type}_{puzzle_number}.txt"
            
            # Write solution
            write_puzzle_txt(output_filename, solution['grid'])
            print(f"Solution written to: {output_filename}")
            
            return solution, decisions, True
        else:
            print("✗ Solution found but validation failed!")
            return solution, decisions, False
    else:
        print("✗ No solution found")
        return None, decisions, False


In [19]:
# Main execution
if __name__ == "__main__":
    solution, decision_count, success = solve_sudoku(PUZZLE_PATH, ALGORITHM, GROUP_ID, PUZZLE_TYPE)
    
    print("\n" + "=" * 50)
    print("FINAL RESULTS")
    print("=" * 50)
    print(f"Algorithm: {ALGORITHM.upper()}")
    print(f"Puzzle: {PUZZLE_PATH}")
    print(f"Decisions: {decision_count}")
    print(f"Success: {success}")
    
    if success and solution is not None:
        print("\nSolution:")
        print(solution['grid'])


Solving Sudoku using GA algorithm
Puzzle: puzzles/Easy-P1.txt
--------------------------------------------------
Puzzle loaded successfully
✗ Solution found but validation failed!

FINAL RESULTS
Algorithm: GA
Puzzle: puzzles/Easy-P1.txt
Decisions: 113936
Success: False


In [20]:
# Main execution
if __name__ == "__main__":
    solution, decisions, success = solve_sudoku(PUZZLE_PATH, ALGORITHM, GROUP_ID, PUZZLE_TYPE)
    
    print("\n" + "=" * 50)
    print("FINAL RESULTS")
    print("=" * 50)
    print(f"Algorithm: {ALGORITHM.upper()}")
    print(f"Puzzle: {PUZZLE_PATH}")
    print(f"Decisions: {decisions}")
    print(f"Success: {success}")
    
    if success and solution is not None:
        print("\nSolution:")
        print(solution)


Solving Sudoku using GA algorithm
Puzzle: puzzles/Easy-P1.txt
--------------------------------------------------
Puzzle loaded successfully
✗ Solution found but validation failed!

FINAL RESULTS
Algorithm: GA
Puzzle: puzzles/Easy-P1.txt
Decisions: 113877
Success: False
